In [101]:
import pandas as pd

In [ ]:
# data_df = pd.read_csv("data/data.csv")
# data_2014 = pd.read_csv("data/data_2014.csv")
data_2014_2024 = pd.read_csv("data/data_2014_2024.csv")
indicator_df = pd.read_csv("data/indicators.csv")
region_df = pd.read_csv("data/regions.csv")
# crime_df = pd.read_csv("data/crime.csv")

In [58]:
# data_df_sample = data_df.sample(n=500000).reset_index()
# data_df_sample.to_csv("data_sample.csv", index=False)

In [59]:
# data_df['date'] = pd.to_datetime(data_df['date'])
# data_2014 = data_df[data_df["date"].dt.year == 2014]
# data_2014.to_csv("data/data_2014.csv", index=False)

In [ ]:
# data_df['date'] = pd.to_datetime(data_df['date'])
# data_2014_2024 = data_df[(data_df["date"].dt.year >= 2014) & (data_df["date"].dt.year <= 2024)]
# data_2014_2024.to_csv("data/data_2014_2024.csv", index=False)

In [61]:
data_2014_2024.head()

,indicator_id,region_id,date,value
0,Z1BR,20,2017-11-30,153373.519224
1,Z1BR,26,2022-02-28,382094.085250
2,Z1BR,53,2022-02-28,224555.043957
3,Z1BR,45,2022-04-30,84132.786751
4,Z1BR,62,2022-10-31,216070.285522


In [62]:
# data_2014
# region_df

In [63]:
data_2014_2024['date'] = pd.to_datetime(data_2014_2024['date'])
data_2014_2024['year'] = data_2014_2024['date'].dt.year
data_2014_2024_clean = data_2014_2024.sort_values(by=["year", "indicator_id", "region_id"], ascending=[False, True, True]).drop_duplicates(subset=["year", "indicator_id", "region_id"], keep="first").reset_index()

In [64]:
data_2014_2024_clean.head()

,index,indicator_id,region_id,date,value,year
0,26285210,CRAM,102001,2024-01-31,0.208119,2024
1,26285939,CRAM,394297,2024-01-31,0.145631,2024
2,26285720,CRAM,394298,2024-01-31,0.225989,2024
3,26285481,CRAM,394299,2024-01-31,0.246032,2024
4,26285986,CRAM,394300,2024-01-31,0.188976,2024


In [65]:
data_2014_2024_clean.tail()

,index,indicator_id,region_id,date,value,year
6173177,4723992,ZSFH,827641,2014-01-31,103651.673074,2014
6173178,4721222,ZSFH,827642,2014-01-31,165813.474380,2014
6173179,4711927,ZSFH,827643,2014-01-31,147200.023145,2014
6173180,4711232,ZSFH,827644,2014-01-31,134829.222175,2014
6173181,4735258,ZSFH,827645,2014-02-28,135779.361110,2014


In [66]:
merged_df = pd.merge(data_2014_2024_clean, region_df, on="region_id", how="left")

In [67]:
merged_df.groupby("region_id").agg("count")

,index,indicator_id,date,value,year,region_type,region
region_id,,,,,,,
3,110,110,110,110,110,110,110
4,110,110,110,110,110,110,110
6,110,110,110,110,110,110,110
8,110,110,110,110,110,110,110
9,110,110,110,110,110,110,110
...,...,...,...,...,...,...,...
845168,96,96,96,96,96,96,96
845169,72,72,72,72,72,72,72
845170,60,60,60,60,60,60,60


In [68]:
check_merge = region_df[region_df["region_id"].isin(data_2014_2024_clean["region_id"])]

In [69]:
check_merge["region_type"].unique()

array(['zip', 'metro', 'state', 'neigh', 'city', 'county'], dtype=object)

In [70]:
check_merge

,region_id,region_type,region
0,96208,zip,"90706;CA;Los Angeles-Long Beach-Anaheim, CA;Be..."
1,394415,metro,"Bridgeport, CT"
2,394653,metro,"Greenville, SC"
3,394312,metro,"Albuquerque, NM"
4,394357,metro,"Bakersfield, CA"
...,...,...,...
89300,59107,zip,"3293;NH;Lebanon, NH-VT;Woodstock;Grafton County"
89301,94027,zip,"83025;WY;Jackson, WY-ID;Wilson;Teton County"
89302,58463,zip,"1718;MA;Boston-Cambridge-Newton, MA-NH;Acton;M..."
89303,75653,zip,40363;KY;nan;nan;Owen County


In [71]:
check_merge_zip = check_merge[check_merge["region_type"] == "zip"].copy()
check_merge_metro = check_merge[check_merge["region_type"] == "metro"].copy()
check_merge_state = check_merge[check_merge["region_type"] == "state"].copy()
check_merge_neigh = check_merge[check_merge["region_type"] == "neigh"].copy()
check_merge_city = check_merge[check_merge["region_type"] == "city"].copy()
check_merge_county = check_merge[check_merge["region_type"] == "county"].copy()

## Splice regions to only get the city and state from them

In [72]:
check_merge_zip.loc[:, "city"] = check_merge_zip["region"].str.split(";").str[3]
check_merge_zip.loc[:, "state"] = check_merge_zip["region"].str.split(";").str[1]

In [73]:
check_merge_zip

,region_id,region_type,region,city,state
0,96208,zip,"90706;CA;Los Angeles-Long Beach-Anaheim, CA;Be...",Bellflower,CA
72,95315,zip,"87121;NM;Albuquerque, NM;Albuquerque;Bernalill...",Albuquerque,NM
73,91325,zip,"76244;TX;Dallas-Fort Worth-Arlington, TX;Fort ...",Fort Worth,TX
74,91732,zip,"77083;TX;Houston-The Woodlands-Sugar Land, TX;...",Houston,TX
75,61616,zip,"10002;NY;New York-Newark-Jersey City, NY-NJ-PA...",New York,NY
...,...,...,...,...,...
89300,59107,zip,"3293;NH;Lebanon, NH-VT;Woodstock;Grafton County",Woodstock,NH
89301,94027,zip,"83025;WY;Jackson, WY-ID;Wilson;Teton County",Wilson,WY
89302,58463,zip,"1718;MA;Boston-Cambridge-Newton, MA-NH;Acton;M...",Acton,MA
89303,75653,zip,40363;KY;nan;nan;Owen County,nan,KY


In [74]:
check_merge_metro

,region_id,region_type,region
1,394415,metro,"Bridgeport, CT"
2,394653,metro,"Greenville, SC"
3,394312,metro,"Albuquerque, NM"
4,394357,metro,"Bakersfield, CA"
5,394308,metro,"Albany, NY"
...,...,...,...
89082,753924,metro,"Urban Honolulu, HI"
89083,395169,metro,"Tulsa, OK"
89084,394619,metro,"Fresno, CA"
89085,395238,metro,"Worcester, MA"


In [75]:
check_merge_metro = check_merge_metro.rename(columns={"region": 'city_state'})
check_merge_metro

,region_id,region_type,city_state
1,394415,metro,"Bridgeport, CT"
2,394653,metro,"Greenville, SC"
3,394312,metro,"Albuquerque, NM"
4,394357,metro,"Bakersfield, CA"
5,394308,metro,"Albany, NY"
...,...,...,...
89082,753924,metro,"Urban Honolulu, HI"
89083,395169,metro,"Tulsa, OK"
89084,394619,metro,"Fresno, CA"
89085,395238,metro,"Worcester, MA"


In [76]:
check_merge_neigh.loc[:, "city"] = check_merge_neigh["region"].str.split(";").str[4]
check_merge_neigh.loc[:, "state"] = check_merge_neigh["region"].str.split(";").str[1]
check_merge_neigh

,region_id,region_type,region,city,state
155,274772,neigh,Northeast Dallas; TX; Dallas-Fort Worth-Arling...,Dallas,TX
164,273698,neigh,"Far North; TX; Dallas-Fort Worth-Arlington, TX...",Dallas,TX
183,275473,neigh,Southeast Dallas; TX; Dallas-Fort Worth-Arling...,Dallas,TX
306,196538,neigh,Murray Hill; NY; New York-Newark-Jersey City; ...,New York,NY
958,118208,neigh,South Los Angeles;CA;Los Angeles-Long Beach-An...,Los Angeles County,CA
...,...,...,...,...,...
88516,807945,neigh,"Dellwood Park;TX;Laredo, TX;Laredo;Webb County",Webb County,TX
88517,806920,neigh,Lake Mathews;CA;Riverside-San Bernardino-Ontar...,Riverside County,CA
88518,250248,neigh,"Central Park;OK;Tulsa, OK;Tulsa;Tulsa County",Tulsa County,OK
88519,416777,neigh,Archwood Acres;TN;Nashville-Davidson--Murfrees...,Davidson County,TN


In [77]:
check_merge_city.loc[:, "city"] = check_merge_city["region"].str.split(";").str[0]
check_merge_city.loc[:, "state"] = check_merge_city["region"].str.split(";").str[1]
check_merge_city

,region_id,region_type,region,city,state
163,32503,city,"Lincoln;NE;Lincoln, NE;Lancaster County",Lincoln,NE
343,17384,city,"Chandler;AZ;Phoenix-Mesa-Chandler, AZ;Maricopa...",Chandler,AZ
344,13437,city,Randolph Township; NJ; New York-Newark-Jersey ...,Randolph Township,NJ
363,47945,city,"Tallahassee;FL;Tallahassee, FL;Leon County",Tallahassee,FL
364,12065,city,"Irving;TX;Dallas-Fort Worth-Arlington, TX;Dall...",Irving,TX
...,...,...,...,...,...
88998,4292,city,"Dalzell;IL;Ottawa, IL;Bureau County",Dalzell,IL
88999,11875,city,Herndon;KS;nan;Rawlins County,Herndon,KS
89000,23854,city,Brownell;KS;nan;Ness County,Brownell,KS
89002,22347,city,New Pine Creek;OR;nan;Lake County,New Pine Creek,OR


In [78]:
check_merge_county.loc[:, "city"] = check_merge_county["region"].str.split(";").str[0]
check_merge_county.loc[:, "state"] = check_merge_county["region"].str.split(";").str[1]
check_merge_county

,region_id,region_type,region,city,state
880,2841,county,San Diego County;CA;San Diego-Chula Vista-Carl...,San Diego County,CA
915,1286,county,Orange County;CA;Los Angeles-Long Beach-Anahei...,Orange County,CA
916,445,county,"Clark County;NV;Las Vegas-Henderson-Paradise, NV",Clark County,NV
918,2049,county,"Tarrant County;TX;Dallas-Fort Worth-Arlington, TX",Tarrant County,TX
932,386,county,"Bexar County;TX;San Antonio-New Braunfels, TX",Bexar County,TX
...,...,...,...,...,...
82375,2400,county,Manistee County;MI;nan,Manistee County,MI
82494,2353,county,"Humboldt County;NV;Winnemucca, NV",Humboldt County,NV
82799,1077,county,Hardin County;IA;nan,Hardin County,IA
82951,263,county,Otoe County;NE;nan,Otoe County,NE


In [79]:
check_merge_combine = pd.concat([check_merge_zip, check_merge_metro, check_merge_neigh, check_merge_city, check_merge_county], ignore_index=True)
check_merge_combine

,region_id,region_type,region,city,state,city_state
0,96208,zip,"90706;CA;Los Angeles-Long Beach-Anaheim, CA;Be...",Bellflower,CA,NaN
1,95315,zip,"87121;NM;Albuquerque, NM;Albuquerque;Bernalill...",Albuquerque,NM,NaN
2,91325,zip,"76244;TX;Dallas-Fort Worth-Arlington, TX;Fort ...",Fort Worth,TX,NaN
3,91732,zip,"77083;TX;Houston-The Woodlands-Sugar Land, TX;...",Houston,TX,NaN
4,61616,zip,"10002;NY;New York-Newark-Jersey City, NY-NJ-PA...",New York,NY,NaN
...,...,...,...,...,...,...
89249,2400,county,Manistee County;MI;nan,Manistee County,MI,NaN
89250,2353,county,"Humboldt County;NV;Winnemucca, NV",Humboldt County,NV,NaN
89251,1077,county,Hardin County;IA;nan,Hardin County,IA,NaN
89252,263,county,Otoe County;NE;nan,Otoe County,NE,NaN


In [80]:
check_merge_combine[check_merge_combine["region_type"] == "metro"]

,region_id,region_type,region,city,state,city_state
31592,394415,metro,NaN,NaN,NaN,"Bridgeport, CT"
31593,394653,metro,NaN,NaN,NaN,"Greenville, SC"
31594,394312,metro,NaN,NaN,NaN,"Albuquerque, NM"
31595,394357,metro,NaN,NaN,NaN,"Bakersfield, CA"
31596,394308,metro,NaN,NaN,NaN,"Albany, NY"
...,...,...,...,...,...,...
32536,753924,metro,NaN,NaN,NaN,"Urban Honolulu, HI"
32537,395169,metro,NaN,NaN,NaN,"Tulsa, OK"
32538,394619,metro,NaN,NaN,NaN,"Fresno, CA"
32539,395238,metro,NaN,NaN,NaN,"Worcester, MA"


In [81]:
merged_df_clean = pd.merge(data_2014_2024_clean, check_merge_combine.drop(["region_type","region"], axis=1), on="region_id", how="inner")
merged_df_clean["date"] = pd.to_datetime(merged_df_clean["date"])
merged_df_clean

,index,indicator_id,region_id,date,value,year,city,state,city_state
0,26285210,CRAM,102001,2024-01-31,0.208119,2024,NaN,NaN,United States
1,36941187,CRAW,102001,2024-01-06,0.070652,2024,NaN,NaN,United States
2,24144778,CRSM,102001,2024-01-31,0.209089,2024,NaN,NaN,United States
3,27231329,CRSW,102001,2024-01-06,0.072053,2024,NaN,NaN,United States
4,37683739,CSAM,102001,2024-01-31,0.196320,2024,NaN,NaN,United States
...,...,...,...,...,...,...,...,...,...
6167567,40087969,ZSFH,422778,2018-01-31,167333.000000,2018,Fort Worth,TX,NaN
6167568,40087957,ZSFH,422778,2017-01-31,159937.000000,2017,Fort Worth,TX,NaN
6167569,40087945,ZSFH,422778,2016-01-31,138216.000000,2016,Fort Worth,TX,NaN
6167570,40087933,ZSFH,422778,2015-01-31,114883.000000,2015,Fort Worth,TX,NaN


In [82]:
states = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY"
}

In [83]:
#crime_df

In [84]:
merged_df_clean["city_state"] = merged_df_clean["city"] + ", " + merged_df_clean["state"]
final_zilow_df = pd.merge(merged_df_clean, indicator_df, on="indicator_id", how="inner")
# old crime dataset stuff that is no longer needed
#crime_df["state_abbrev"] = crime_df["State"].map(states)
#crime_df["city_state"] = crime_df["City"] + ", " + crime_df["state_abbrev"]
#crime_df_max = crime_df.groupby(["city_state", "Year", "Month"])["Incident"].max().reset_index()
#crime_df_sum = crime_df_max.groupby("city_state")["Incident"].sum()
#final_df = pd.merge(final_zilow_df, crime_df_sum.rename("total_incidents"), on="city_state", how="inner")

In [85]:
final_zilow_df["year_city_state"] = final_zilow_df["date"].dt.year.astype(str) + ": " + final_zilow_df["city_state"]
final_zilow_df

,index,indicator_id,region_id,date,value,year,city,state,city_state,indicator,category,year_city_state
0,26285210,CRAM,102001,2024-01-31,0.208119,2024,NaN,NaN,NaN,"Share of Listings With a Price Cut (Raw, All H...",Inventory and sales,NaN
1,26273544,CRAM,102001,2023-01-31,0.214597,2023,NaN,NaN,NaN,"Share of Listings With a Price Cut (Raw, All H...",Inventory and sales,NaN
2,26261917,CRAM,102001,2022-01-31,0.113713,2022,NaN,NaN,NaN,"Share of Listings With a Price Cut (Raw, All H...",Inventory and sales,NaN
3,26250392,CRAM,102001,2021-01-31,0.120474,2021,NaN,NaN,NaN,"Share of Listings With a Price Cut (Raw, All H...",Inventory and sales,NaN
4,26238562,CRAM,102001,2020-01-31,0.173635,2020,NaN,NaN,NaN,"Share of Listings With a Price Cut (Raw, All H...",Inventory and sales,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
6167567,45111107,RSSA,394521,2018-01-31,932.000000,2018,NaN,NaN,NaN,"ZORI (Smoothed, Seasonally Adjusted): All Home...",Rentals,NaN
6167568,45109485,RSSA,394521,2017-01-31,896.000000,2017,NaN,NaN,NaN,"ZORI (Smoothed, Seasonally Adjusted): All Home...",Rentals,NaN
6167569,45107988,RSSA,394521,2016-01-31,870.000000,2016,NaN,NaN,NaN,"ZORI (Smoothed, Seasonally Adjusted): All Home...",Rentals,NaN
6167570,45106587,RSSA,394521,2015-01-31,840.000000,2015,NaN,NaN,NaN,"ZORI (Smoothed, Seasonally Adjusted): All Home...",Rentals,NaN


In [86]:
# no longer needed since we found a better dataset
# final_df.to_csv("data/final_df.csv", index=False)

In [87]:
better_crime_df = pd.read_csv("data/better_crime.csv")

C:\Users\zacha\AppData\Local\Temp\ipykernel_30812\1849724017.py:1: DtypeWarning: Columns (19,20,25,50) have mixed types. Specify dtype option on import or set low_memory=False.
  better_crime_df = pd.read_csv("data/better_crime.csv")


In [88]:
better_crime_df.head()

,ori,ori9,agency_name,state,state_abb,year,number_of_months_missing,number_of_months_reported,last_month_reported,fips_state_code,...,unfounded_motor_vehicle_theft_total,unfounded_motor_vehicle_theft_car,unfounded_motor_vehicle_theft_truck,unfounded_motor_vehicle_theft_other,unfounded_all_crimes,unfounded_theft_under50_dollar,unfounded_assault_aggravated,unfounded_index_violent,unfounded_index_property,unfounded_index_total
0,AK00101,AK0010100,anchorage,alaska,AK,2024,NaN,12,december,2.0,...,31,22,7,2,91,0,6,13,67,80
1,AK00101,AK0010100,anchorage,alaska,AK,2023,NaN,12,december,2.0,...,40,33,6,1,144,0,12,22,97,119
2,AK00101,AK0010100,anchorage,alaska,AK,2022,NaN,12,december,2.0,...,52,40,12,0,99,0,2,12,87,99
3,AK00101,AK0010100,anchorage,alaska,AK,2021,NaN,0,no months reported,2.0,...,0,0,0,0,0,0,0,0,0,0
4,AK00101,AK0010100,anchorage,alaska,AK,2020,NaN,12,december,2.0,...,37,22,15,0,89,0,0,5,84,89


In [89]:
better_crime_df = better_crime_df[better_crime_df["address_city"].str.len() > 2]
better_crime_df["address_city"].value_counts()

address_city
harrisburg       7697
indianapolis     5932
columbia         5875
albany           4517
tallahassee      3943
                 ... 
krugerville        13
felicity           13
walthourville      13
chimney rock       13
loving             13
Name: count, Length: 9034, dtype: int64

In [90]:
crime_2014_2024 = better_crime_df[(better_crime_df["year"] >= 2014) & (better_crime_df["year"] <= 2024)]
crime_2014_2024.to_csv("data/better_crime_subset.csv", index=False)

In [91]:
crime_2014_2024["city_state"] = crime_2014_2024["address_city"].str.capitalize() + ", " + crime_2014_2024["address_state"].str.upper()
crime_2014_2024["year_city_state"] = crime_2014_2024["year"].astype(str) + ": " + crime_2014_2024["city_state"]

C:\Users\zacha\AppData\Local\Temp\ipykernel_30812\3352993290.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_2014_2024["city_state"] = crime_2014_2024["address_city"].str.capitalize() + ", " + crime_2014_2024["address_state"].str.upper()
C:\Users\zacha\AppData\Local\Temp\ipykernel_30812\3352993290.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  crime_2014_2024["year_city_state"] = crime_2014_2024["year"].astype(str) + ": " + crime_2014_2024["city_state"]


In [92]:
crime_2014_2024_clean = crime_2014_2024[["actual_murder", "actual_rape_total", "actual_robbery_total", "actual_assault_total", "actual_theft_total", "year", "year_city_state"]].copy()
crime_2014_2024_clean

,actual_murder,actual_rape_total,actual_robbery_total,actual_assault_total,actual_theft_total,year,year_city_state
0,27,462,479,6459,5813,2024,"2024: Anchorage, AK"
1,22,428,461,6620,6078,2023,"2023: Anchorage, AK"
2,29,492,436,6530,5605,2022,"2022: Anchorage, AK"
3,0,0,0,0,0,2021,"2021: Anchorage, AK"
4,18,558,558,5777,7279,2020,"2020: Anchorage, AK"
...,...,...,...,...,...,...,...
1137271,0,0,0,0,0,2018,"2018: Cheyenne, WY"
1137272,0,0,0,0,0,2017,"2017: Cheyenne, WY"
1137273,0,0,0,0,0,2016,"2016: Cheyenne, WY"
1137274,0,0,0,0,0,2015,"2015: Cheyenne, WY"


In [94]:
final_df = pd.merge(final_zilow_df, crime_2014_2024_clean, on="year_city_state", how="inner")
final_df_sorted = final_df.sort_values(by=["date"], ascending=[False]).reset_index()

In [100]:
# final_df.to_csv("data/final.csv", index=False)
# final_df_sorted.to_csv("data/final_sorted.csv", index=False)

In [95]:
final_df.shape

(5805583, 18)

In [96]:
final_df_sorted.head()

,level_0,index,indicator_id,region_id,date,value,year_x,city,state,city_state,indicator,category,year_city_state,actual_murder,actual_rape_total,actual_robbery_total,actual_assault_total,actual_theft_total,year_y
0,5796266,30490398,ZSFH,85286,2024-07-31,159660.353250,2024,Hardin,IL,"Hardin, IL",ZHVI Single-Family Homes Time Series ($),Home values,"2024: Hardin, IL",0,0,0,44,8,2024
1,5505832,30483034,ZSFH,90672,2024-07-31,337652.333964,2024,Pottsboro,TX,"Pottsboro, TX",ZHVI Single-Family Homes Time Series ($),Home values,"2024: Pottsboro, TX",0,2,0,8,8,2024
2,2046859,30480306,ZSFH,63963,2024-07-31,597961.962978,2024,Pittsburgh,PA,"Pittsburgh, PA",ZHVI Single-Family Homes Time Series ($),Home values,"2024: Pittsburgh, PA",60,195,17,379,112,2024
3,2046858,30480306,ZSFH,63963,2024-07-31,597961.962978,2024,Pittsburgh,PA,"Pittsburgh, PA",ZHVI Single-Family Homes Time Series ($),Home values,"2024: Pittsburgh, PA",2,9,20,225,165,2024
4,2046857,30480306,ZSFH,63963,2024-07-31,597961.962978,2024,Pittsburgh,PA,"Pittsburgh, PA",ZHVI Single-Family Homes Time Series ($),Home values,"2024: Pittsburgh, PA",0,1,0,56,54,2024


In [97]:
final_df_sorted.tail()

,level_0,index,indicator_id,region_id,date,value,year_x,city,state,city_state,indicator,category,year_city_state,actual_murder,actual_rape_total,actual_robbery_total,actual_assault_total,actual_theft_total,year_y
5805578,2190293,41769430,Z5BR,63972,2014-01-31,450096.569504,2014,Pittsburgh,PA,"Pittsburgh, PA",ZHVI 5+ Bedroom Time Series ($),Home values,"2014: Pittsburgh, PA",0,0,1,7,59,2014
5805579,2190292,41769430,Z5BR,63972,2014-01-31,450096.569504,2014,Pittsburgh,PA,"Pittsburgh, PA",ZHVI 5+ Bedroom Time Series ($),Home values,"2014: Pittsburgh, PA",0,0,0,7,51,2014
5805580,2190291,41769430,Z5BR,63972,2014-01-31,450096.569504,2014,Pittsburgh,PA,"Pittsburgh, PA",ZHVI 5+ Bedroom Time Series ($),Home values,"2014: Pittsburgh, PA",0,0,0,0,11,2014
5805581,2190290,41769430,Z5BR,63972,2014-01-31,450096.569504,2014,Pittsburgh,PA,"Pittsburgh, PA",ZHVI 5+ Bedroom Time Series ($),Home values,"2014: Pittsburgh, PA",0,1,0,7,19,2014
5805582,2193167,40750570,ZCON,63960,2014-01-31,104533.876063,2014,Pittsburgh,PA,"Pittsburgh, PA",ZHVI Condo/Co-op Time Series ($),Home values,"2014: Pittsburgh, PA",0,0,0,0,6,2014
